<a href="https://colab.research.google.com/github/diego-minguzzi/reinforcement-learning/blob/master/optuna_hyperparams_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyperparameter optimization for Deep RL using Optuma
Diego Minguzzi ([profile](https://www.linkedin.com/in/diego-minguzzi-2775b78/))

Online course [Deep Reinforcement Learning](https://huggingface.co/learn/deep-rl-course), by Hugging Face: exercise of the [Bonus Unit 2](https://huggingface.co/learn/deep-rl-course/unitbonus2/introduction).
<br>
Adapted from Antonin’s Raffin ICRA 2022 presentations:

*  [The tutorial on YouTube](https://youtu.be/ihP7E76KGOI)
*  [The related Colab Notebook](https://colab.research.google.com/github/araffin/tools-for-robotic-rl-icra2022/blob/main/notebooks/optuna_lab.ipynb)


See [Optuna](https://optuna.org/) and the [documentation](https://optuna.readthedocs.io/en/stable/index.html)

The Gym Lunar lander environment is used.<br>

## Lunar Lander environment

## Imports and package installation

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt install swig cmake
!pip install gymnasium[box2d]
!pip install huggingface_sb3
!pip install stable-baselines3
!pip install sb3-contrib
!pip install optuna
!pip install optuna-dashboard
!pip install jupyterlab jupyterlab-optuna

In [3]:
import gymnasium as gym
import collections
import json
import math
import numpy as np
import os.path
import pickle
import torch
from typing import Any, Dict
import torch.nn as nn

In [4]:
from stable_baselines3 import PPO, A2C, SAC, TD3, DQN
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import BaseCallback, CallbackList

In [5]:
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_intermediate_values

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Set all parameters

In [6]:
import logging as log
LOG_FORMAT_STRING = "%(asctime)s [%(levelname)-7s] %(message)s"
LOG_LEVEL= log.DEBUG
LOG_ROOT_LOGGER = ''

N_TRIALS =          25  # Maximum number of trials
N_JOBS =            1   # Number of jobs to run in parallel
N_STARTUP_TRIALS=   3   # Stop random sampling after N_STARTUP_TRIALS
N_EVALUATIONS=      6
N_TIMESTEPS =       int(300000)  # Training budget
EVAL_FREQ =         int(N_TIMESTEPS / N_EVALUATIONS)
N_EVAL_ENVS =       4
N_EVAL_EPISODES =   8
TIMEOUT =           int(60 * 60 * 5) # Expressed into seconds.
NUM_STEPS_LR_UPDATE=5000 # After this many steps the learning rate is updated
LR_UPDATE_FACTOR =  0.95 # Learning rate update factor
POLICY=             'MlpPolicy'
IS_DETERMINISTIC=   True
IS_VERBOSE=         0
N_WARMUP_STEPS=     N_EVALUATIONS // 3  # Do not prune before the warmup steps.
SAVED_PARAMS_FILE=  'lunar_lander_params.json'
SAVED_SNAPSHOT_FILE='opt_lunar_lander_study.pkl'
SEED =              2024
ENV_ID =            'LunarLander-v2'
BEST_MODEL_FILE=    f'{ENV_ID}.model'
RESULT_OBJECTIVE=   375.0

HyperParams= collections.namedtuple('HyperParams', ['gamma',
                                                    'learning_rate',
                                                    'max_grad_norm',
                                                    'n_epochs',
                                                    'n_steps',
                                                    'ent_coef',
                                                    'batch_size'])
# Guess Hyperparameters, used in the trials after GUESS_HYPERPARAMS_AFTER_N_TRIAL
guess_hyperparams=[ HyperParams(gamma = 1.0 - 0.0016617315216418454,
                                learning_rate = math.pow(10., -2.682161433845602),
                                max_grad_norm = 0.3733377073391161,
                                n_epochs= 5,
                                n_steps = 2 ** 12,
                                ent_coef= 0.01,
                                batch_size=64),
                    HyperParams(gamma = 1.0 - 0.00010534109595607163,
                                learning_rate = math.pow(10., -2.7388359550396095),
                                max_grad_norm = 0.30386280137092286,
                                n_epochs= 5,
                                n_steps = 2 ** 12,
                                ent_coef= 0.01,
                                batch_size=64)
]

GUESS_HYPERPARAMS_AFTER_N_TRIAL=10

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
log.basicConfig(level=LOG_LEVEL, format=LOG_FORMAT_STRING)
log.getLogger().setLevel(LOG_LEVEL)
roothandler= log.getLogger().handlers[0]
roothandler.setFormatter( log.Formatter(LOG_FORMAT_STRING) )

In [8]:
DEFAULT_HYPERPARAMS = {
    "policy": POLICY,
    "env": ENV_ID,
}

In [9]:
np.random.seed(SEED)

In [10]:
class DecreaseLearningRateCallback(BaseCallback):
    """ Custom callback to decrease the learning rate periodically during training. """

    def __init__(self, decay_interval, decay_factor):
        super(DecreaseLearningRateCallback, self).__init__()
        self.decay_interval = decay_interval  # Interval at which to decrease the learning rate
        self.decay_factor = decay_factor      # Factor by which to decrease the learning rate

    def _on_step(self) -> bool:
        if 0==(self.num_timesteps % self.decay_interval):
            # Retrieve the current learning rate
            current_lr = self.model.learning_rate

            # Decrease the learning rate by the decay factor
            new_lr = current_lr * self.decay_factor

            # Set the new learning rate
            self.model.learning_rate = new_lr
            if 0==(self.num_timesteps % (10*self.decay_interval)):
              log.debug(f'Num steps:{self.num_timesteps} current_lr:{current_lr} new_lr:{new_lr}')

        return True

### Define the sampling function



In [11]:
def sample_ppo_params(trial: optuna.Trial) -> Dict[str, Any]:
    """ Samples the model hyperparameters.
    :param trial: Optuna trial object
    :return: The sampled hyperparameters for the given trial.
    """
    indx_trial= trial.number
    if ( (indx_trial>=GUESS_HYPERPARAMS_AFTER_N_TRIAL)
      and (indx_trial - GUESS_HYPERPARAMS_AFTER_N_TRIAL) < len(guess_hyperparams) ):
      indx_guess= indx_trial - GUESS_HYPERPARAMS_AFTER_N_TRIAL
      log.info(f'indx_trial:{indx_guess} indx_guess:{indx_guess}: using guessed params.')
      gamma = guess_hyperparams[indx_guess].gamma
      learning_rate = guess_hyperparams[indx_guess].learning_rate
      max_grad_norm = guess_hyperparams[indx_guess].max_grad_norm
      n_epochs = guess_hyperparams[indx_guess].n_epochs
      n_steps = guess_hyperparams[indx_guess].n_steps
      ent_coef= guess_hyperparams[indx_guess].ent_coef
      batch_size= guess_hyperparams[indx_guess].batch_size
    else:
      gamma = 1.0 - trial.suggest_float("gamma", 0.0001, 0.2, log=True)
      learning_rate = math.pow(10., trial.suggest_float("learning_rate_exp", -4.0, -1.0))
      max_grad_norm = trial.suggest_float("max_grad_norm", 0.3, 10.0, log=True)
      n_epochs= trial.suggest_int("n_epochs", 2, 16)
      n_steps = 2 ** trial.suggest_int("exponent_n_steps", 6, 13)

    ent_coef= guess_hyperparams[-1].ent_coef
    batch_size= guess_hyperparams[-1].batch_size

    result= {
        "ent_coef": ent_coef,
        "n_steps": n_steps,
        "gamma": gamma,
        "learning_rate": learning_rate,
        "max_grad_norm": max_grad_norm,
        "n_epochs":n_epochs,
        "batch_size":batch_size
    }
    log.info(f'sample_ppo_params() {trial.number}\nParams:{result}')
    return result

### Integrate the performance evaluation in stable baselines.

In [12]:
from stable_baselines3.common.callbacks import EvalCallback

class TrialEvalCallback(EvalCallback):
    """
    Callback used for evaluating and reporting a trial.

    :param eval_env: Evaluation environement
    :param trial: Optuna trial object
    :param n_eval_episodes: Number of evaluation episodes
    :param eval_freq:   Evaluate the agent every ``eval_freq`` call of the callback.
    :param deterministic: Whether the evaluation should
        use a stochastic or deterministic policy.
    :param verbose:
    """
    def __init__(self,
                  eval_env: gym.Env,
                  trial: optuna.Trial,
                  n_eval_episodes: int = 5,
                  eval_freq: int = 10000,
                  deterministic: bool = IS_DETERMINISTIC,
                  verbose: int = IS_VERBOSE):

        super().__init__(
            eval_env=eval_env,
            n_eval_episodes=n_eval_episodes,
            eval_freq=eval_freq,
            deterministic=deterministic,
            verbose=verbose,
        )
        self.trial = trial
        self.eval_idx = 0
        self.is_pruned = False

    def _on_step(self) -> bool:
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            # Evaluate policy (done in the parent class)
            super()._on_step()
            self.eval_idx += 1
            # Send report to Optuna
            self.trial.report(self.last_mean_reward, self.eval_idx)
            # Prune trial if need
            if self.trial.should_prune():
                self.is_pruned = True
                return False
        return True

### Define the objective function: it evaluates a set of hyperparameters.

In [13]:
def objective(trial: optuna.Trial) -> float:
    """
    Objective function using by Optuna to evaluate
    one configuration (i.e., one set of hyperparameters).

    Given a trial object, it will sample hyperparameters,
    evaluate it and report the result (mean episodic reward after training)

    :param trial: Optuna trial object
    :return: Mean episodic reward after training
    """
    print(f'objective started trial:{trial.number}.')
    kwargs = DEFAULT_HYPERPARAMS.copy()

    # 1. Sample hyperparameters and update the keyword arguments
    sampled_params= sample_ppo_params( trial)
    kwargs.update( sampled_params)

    # Create the RL model
    model = PPO(**kwargs)

    # 2. Create envs used for evaluation using `make_vec_env`, `ENV_ID` and `N_EVAL_ENVS`
    eval_env= make_vec_env(ENV_ID, N_EVAL_ENVS)

    # 3. Create the `TrialEvalCallback` callback defined above that will periodically evaluate
    eval_callback = TrialEvalCallback( eval_env, trial, N_EVAL_EPISODES, EVAL_FREQ, deterministic=IS_DETERMINISTIC)
    lr_callback = DecreaseLearningRateCallback(decay_interval=NUM_STEPS_LR_UPDATE, decay_factor=LR_UPDATE_FACTOR)
    list_callbacks = CallbackList([eval_callback, lr_callback])

    nan_encountered = False
    try:
        # Train the model
        model.learn(N_TIMESTEPS, callback=list_callbacks)
    except AssertionError as e:
        # Sometimes, random hyperparams can generate NaN
        log.error('Exception in objective:{e}')
        nan_encountered = True
    finally:
        # Free memory
        model.env.close()
        eval_env.close()

    # Tell the optimizer that the trial failed
    if nan_encountered:
        return float("nan")

    if eval_callback.is_pruned:
        raise optuna.exceptions.TrialPruned()

    mean_reward, std_reward = evaluate_policy(model, eval_env, N_EVAL_EPISODES, deterministic=IS_DETERMINISTIC)
    result = mean_reward-std_reward
    log.info(f'objective: trial:{trial.number}\nMean reward:{mean_reward} std_reward:{std_reward} result:{result}')

    if (0==trial.number) or ( (trial.number>0) and (result > trial.study.best_value) ):
      model.save(BEST_MODEL_FILE)
      log.info(f'Saved best model to file:{BEST_MODEL_FILE}')

    eval_callback.mean_minus_std= result
    return result

### Run the Optimization loop

The optimization loop runs for a given number of trials, or until the TIMEOUT expires.<br>
Then, a snapshot file is saved.  If the snapshot exists, it is loaded before starting the optimization.

In [14]:
import torch as th

# Set pytorch num threads to 1 for faster training
th.set_num_threads(1)

# Select the sampler, can be random, TPESampler, CMAES, ...
sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)

pruner = MedianPruner( n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_WARMUP_STEPS )

# Create the study and start the hyperparameter optimization
if not os.path.exists(SAVED_SNAPSHOT_FILE):
  log.info('Started the study from scratch.')
  study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")
else:
  log.info(f'Resuming the study from the file: {SAVED_SNAPSHOT_FILE}')
  snapshot_file = open(SAVED_SNAPSHOT_FILE, 'rb')
  with open(SAVED_SNAPSHOT_FILE, 'rb') as snapshot_file:
    study = pickle.load(snapshot_file)

try:
  study.optimize(objective, n_trials=N_TRIALS, n_jobs=N_JOBS, timeout=TIMEOUT)
except Exception as e:
  log.error(f'Exception during study: {e}')
except KeyboardInterrupt:
    pass

snapshot_file = open(SAVED_SNAPSHOT_FILE, 'wb')
pickle.dump(study, snapshot_file)
snapshot_file.close()

log.info(f'Saved the study to the file: {os.path.abspath(SAVED_SNAPSHOT_FILE)}')

2024-01-01 08:42:49,860 [INFO   ] Started the study from scratch.
[I 2024-01-01 08:42:49,862] A new study created in memory with name: no-name-016626b3-0f0b-47de-8ca5-9f23c1fa928f
2024-01-01 08:42:49,871 [INFO   ] sample_ppo_params() 0
Params:{'ent_coef': 0.01, 'n_steps': 1024, 'gamma': 0.9947205391014854, 'learning_rate': 0.0042755129544146675, 'max_grad_norm': 3.5955466505396703, 'n_epochs': 8, 'batch_size': 64}


objective started trial:0.


2024-01-01 08:45:18,801 [DEBUG  ] Num steps:50000 current_lr:0.002694639515789764 new_lr:0.002559907540000276
2024-01-01 08:47:54,833 [DEBUG  ] Num steps:100000 current_lr:0.00161338021603475 new_lr:0.0015327112052330124
2024-01-01 08:50:20,383 [DEBUG  ] Num steps:150000 current_lr:0.0009659903323764005 new_lr:0.0009176908157575804
2024-01-01 08:52:34,980 [DEBUG  ] Num steps:200000 current_lr:0.00057837409494091 new_lr:0.0005494553901938645
2024-01-01 08:55:06,590 [DEBUG  ] Num steps:250000 current_lr:0.00034629393533968793 new_lr:0.0003289792385727035
2024-01-01 08:57:32,397 [DEBUG  ] Num steps:300000 current_lr:0.00020733897092209777 new_lr:0.00019697202237599286
2024-01-01 08:57:35,022 [INFO   ] objective: trial:0
Mean reward:-94.604606875 std_reward:91.11242539314406 result:-185.71703226814407
2024-01-01 08:57:35,039 [INFO   ] Saved best model to file:LunarLander-v2.model
[I 2024-01-01 08:57:35,044] Trial 0 finished with value: -185.71703226814407 and parameters: {'gamma': 0.005279

objective started trial:1.


2024-01-01 08:59:19,161 [DEBUG  ] Num steps:50000 current_lr:0.00037118249628686947 new_lr:0.00035262337147252596
2024-01-01 09:01:45,950 [DEBUG  ] Num steps:100000 current_lr:0.000222240671725661 new_lr:0.00021112863813937794
2024-01-01 09:04:22,531 [DEBUG  ] Num steps:150000 current_lr:0.00013306369956330355 new_lr:0.00012641051458513837
2024-01-01 09:07:07,463 [DEBUG  ] Num steps:200000 current_lr:7.967015220026753e-05 new_lr:7.568664459025415e-05
2024-01-01 09:09:50,966 [DEBUG  ] Num steps:250000 current_lr:4.770146307704395e-05 new_lr:4.531638992319175e-05
2024-01-01 09:12:03,686 [DEBUG  ] Num steps:300000 current_lr:2.856062799994182e-05 new_lr:2.713259659994473e-05
2024-01-01 09:12:11,255 [INFO   ] objective: trial:1
Mean reward:132.43273125000002 std_reward:113.80482462081112 result:18.6279066291889
2024-01-01 09:12:11,266 [INFO   ] Saved best model to file:LunarLander-v2.model
[I 2024-01-01 09:12:11,267] Trial 1 finished with value: 18.6279066291889 and parameters: {'gamma': 0

...

2024-01-01 12:26:44,261 [DEBUG  ] Num steps:50000 current_lr:0.0005320924350190845 new_lr:0.0005054878132681302
2024-01-01 12:29:33,222 [DEBUG  ] Num steps:100000 current_lr:0.00031858339593522254 new_lr:0.0003026542261384614
2024-01-01 12:32:15,155 [DEBUG  ] Num steps:150000 current_lr:0.00019074764737442363 new_lr:0.00018121026500570245
2024-01-01 12:34:59,309 [DEBUG  ] Num steps:200000 current_lr:0.00011420766255588396 new_lr:0.00010849727942808976
2024-01-01 12:37:41,715 [DEBUG  ] Num steps:250000 current_lr:6.838034631627956e-05 new_lr:6.496132900046557e-05
2024-01-01 12:40:20,667 [DEBUG  ] Num steps:300000 current_lr:4.0941839257469564e-05 new_lr:3.889474729459608e-05
2024-01-01 12:40:35,563 [INFO   ] objective: trial:22
Mean reward:215.149437875 std_reward:53.2343373591899 result:161.91510051581008
[I 2024-01-01 12:40:35,565] Trial 22 finished with value: 161.91510051581008 and parameters: {'gamma': 0.0016243900310192799, 'learning_rate_exp': -3.0735253631774864, 'max_grad_norm'

objective started trial:23.


2024-01-01 12:43:02,911 [DEBUG  ] Num steps:50000 current_lr:0.001993329476107443 new_lr:0.0018936630023020705
2024-01-01 12:45:39,348 [DEBUG  ] Num steps:100000 current_lr:0.0011934799894182109 new_lr:0.0011338059899473002
2024-01-01 12:48:08,812 [DEBUG  ] Num steps:150000 current_lr:0.0007145805559065118 new_lr:0.0006788515281111862
[I 2024-01-01 12:48:08,814] Trial 23 pruned. 
2024-01-01 12:48:08,840 [INFO   ] sample_ppo_params() 24
Params:{'ent_coef': 0.01, 'n_steps': 4096, 'gamma': 0.990154232513451, 'learning_rate': 0.0002501465694649141, 'max_grad_norm': 3.678972182557771, 'n_epochs': 9, 'batch_size': 64}


objective started trial:24.


2024-01-01 12:50:10,440 [DEBUG  ] Num steps:50000 current_lr:0.00015765472774989804 new_lr:0.00014977199136240313
2024-01-01 12:52:48,249 [DEBUG  ] Num steps:100000 current_lr:9.439370914943381e-05 new_lr:8.967402369196211e-05
[I 2024-01-01 12:52:48,251] Trial 24 pruned. 
2024-01-01 12:52:48,254 [INFO   ] Saved the study to the file: /content/opt_lunar_lander_study.pkl


In [15]:
# Write report
study.trials_dataframe().to_csv("study_results_ppo_lunar_lander.csv")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
fig1 = plot_optimization_history(study)
fig2 = plot_param_importances(study)
fig3 = plot_intermediate_values(study)

fig1.show()
fig2.show()
fig3.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
with open(SAVED_PARAMS_FILE, 'w') as fp:
    json.dump(study.best_params, fp)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [18]:
study.best_params

{}

In [19]:
if len(study.trials)>0:
    trial = study.best_trial
    print(f'Best trial: {trial.number} Value: {trial.value}')

    print("  Params: ")
    for key, value in study.best_params.items():
        print(f"    {key}: {value}")

Best trial: 10 Value: 239.74346462801856
  Params: 
